In [145]:
import time
import math
import numpy as np
import pandas as pd
import scipy as sc
from scipy.optimize import minimize 
from scipy.optimize import Bounds

# Чтение данных

In [146]:
data = pd.read_csv("data/train.csv", delimiter=',')

In [147]:
data_test = pd.read_csv("data/test.csv", delimiter=',')

In [148]:
size_test = data_test.shape[0]
size_test

1037875

In [149]:
data.head()

,Password,Times
0,631XniVx2lS5I,2
1,LEGIT747,1
2,742364es,1
3,3846696477,1
4,laurahop,2


In [150]:
arr_a = np.array(list(map(float, data['Times'].values)))

In [151]:
arr_a[0:5]

array([2., 1., 1., 1., 2.])

In [152]:
arr_len = len(arr_a)
arr_len

4151496

# Расчет порога с использованием методов оптимизации

$$ rmsle = \sqrt{ \frac{1}{n} \sum_{i=1}^{n} (log(p_i + 1) - log(a_i + 1)) ^ 2  }  $$

$p_i$ - predicted

$a_i$ - actual

In [153]:
def f_rmsle(arr_p, arr_a, arr_len):
    """
    Функция для вычисления метрики 
    """
    lst = [(math.log(p+1)-math.log(a+1))**2 for p,a in zip(arr_p, arr_a)]
    lst_sum = sum(lst)
    res = math.sqrt(lst_sum / arr_len)
    return res

In [154]:
def f_minimize(arr_const):
    """
    Функция для вычисления в minimize
    """
    const = arr_const[0]
    arr_p = np.full_like(arr_a, const)
    assert len(arr_p) == len(arr_a)
    return f_rmsle(arr_p, arr_a, arr_len)

In [127]:
init = np.array([1.3])
init

array([1.3])

In [128]:
bnds = Bounds(np.array([0.5]), np.array([2.3]), keep_feasible=True)
bnds

In [129]:
%%time
arr_const_minimize = minimize(f_minimize, init, bounds=bnds)

Wall time: 10min 44s


In [130]:
arr_const_minimize

      fun: 0.4092226113650883
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.0011029])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 82
      nit: 4
   status: 0
  success: True
        x: array([1.26337675])

In [134]:
opt_const = arr_const_minimize.x[0]
opt_const

1.2633767456739853

# Сохранение результатов

In [143]:
def save_res_const(cnst, fn):
    y_pred = np.array([cnst]*size_test)
    df_res = data_test[['Id']]
    df_res['Times'] = y_pred
    df_res.to_csv('data/res/' + fn + '.csv', sep=',', index=None)

In [144]:
save_res_const(opt_const, 'const1')

# Score = 0.41231